In [ ]:
import pandas as pd 

import os 

from parc import PARC

In [ ]:

columnvalues_to_keep = ['89Y_CD45', 
                #'102Pd_BC1', 
                #'103Rh', 
                #'104Pd_BC2', 
                #'105Pd_BC3',
                # '106Pd_BC4', 
                # '108Pd_BC5', 
                # '110Pd_BC6', 
                '111Cd_CD3', 
                '112Cd_CD34',
                '113Cd_CD123', 
                '114Cd_CD66b', 
                #'115In', 
                '116Cd_HLA-DR', 
                # '120Sn', 
                # '127I',
                # '131Xe', 
                # '133Cs', 
                # '136Ba', 
                # '137Ba', 
                # '138Ba', 
                # '139La', 
                # '140Ce',
                '141Pr_CD38', 
                '142Nd_cCaspase3', 
                '143Nd_pCRKL Y207', 
                '144Nd_pTyr',
                '145Nd_CD4', 
                '146Nd_CD49d', 
                '147Sm_CD20', 
                '148Nd_CD16', 
                '149Sm_CD25',
                '150Nd_pSTAT5 Y694', 
                '151Eu_pSTAT3 S727', 
                '152Sm_CD13',
                '153Eu_pSTAT1 Y701', 
                '154Sm_CD45RA', 
                '155Gd_CD27',
                '156Gd_pp38 T180Y182', 
                '157Gd_CD8a', 
                '158Gd_pSTAT3 Y705',
                '159Tb_pMAPKAPK T334', 
                '160Gd_CD14', 
                '161Dy_CD26', 
                '162Dy_FoxP3',
                '163Dy_CD56', 
                '164Dy_CD15', 
                '165Ho_pCREB S133', 
                '166Er_MPO',
                '167Er_IL1-RAP', 
                '168Er_CD117', 
                '169Tm_CD33', 
                '170Er_pSRC Y418',
                '171Yb_pERK T202Y204', 
                '172Yb_pS6 S235S236', 
                '173Yb_STAT3tot',
                '174Yb_CD11c', 
                '175Lu_CXCR4', 
                '176Yb_pS6 S240244', 
                #'191Ir_DNA1',
                #'193Ir_DNA2', 
                #'194Pt_mBC1', 
                '195Pt_mBC2', 
                #'196Pt_mBC3', 
                #'198Pt_mBC4',
                #'208Pb', 
                '209Bi_CD11b']    
        

external = ['89Y_CD45', 
                '111Cd_CD3', 
                '112Cd_CD34',
                '113Cd_CD123', 
                '114Cd_CD66b', 
                '116Cd_HLA-DR', 
                '141Pr_CD38', 
                '145Nd_CD4', 
                '146Nd_CD49d', 
                '147Sm_CD20', 
                '148Nd_CD16', 
                '149Sm_CD25',
                '152Sm_CD13',
                '154Sm_CD45RA', 
                '155Gd_CD27', 
                '157Gd_CD8a', 
                '160Gd_CD14', 
                '161Dy_CD26', 
                '162Dy_FoxP3',
                '163Dy_CD56', 
                '164Dy_CD15', 
                '166Er_MPO',
                '167Er_IL1-RAP', 
                '168Er_CD117', 
                '169Tm_CD33',  
                '174Yb_CD11c', 
                '175Lu_CXCR4',
                '195Pt_mBC2',
                '209Bi_CD11b']

### Foreslått test av SEG

Les inn alle 12 filene og kjør clustering på interne og eksterne kanaler. (det er litt uortodoks, hva mener du? SEG VIL KLUSTRE MED DE INTRACELLULÆRE OGS, FORDI VI SKAL MÅLE CV I ALLE CLUSTRENE OG VI HAR IKKE MED DE INTRACELLULÆRE, SÅ HAR VI INGEN BATCHEFFEKT PÅ VÅR ANALYSE)

La oss si dette genererer 30 clusters.

In [ ]:
data_batch_1_4 = pd.read_pickle('../cytofdata/uncorrected_batch_1_4.pkl')

df_rt_rj = data_batch_1_4[data_batch_1_4['file_id'].isin(['RT', 'RJ'])]

In [ ]:
df_rt_rj

In [ ]:
clustered_data_file = 'clusters/rjrt_uncorr_1_4.pkl'

if os.path.exists(clustered_data_file):
    print('Cluster found, loading from file')
    
    df_rt_rj = pd.read_pickle(clustered_data_file)
    
else:
    print('No saved cluster found, running PARC')

    data_for_clustering = df_rt_rj[columnvalues_to_keep]

    parc = PARC(data_for_clustering.values, 
                resolution_parameter=0.5,
                knn = 30, 
                small_pop = 100,
                hnsw_param_ef_construction = 2000)

    parc.run_PARC()

    clusters = parc.labels
    df_rt_rj['cluster'] = clusters  

    df_rt_rj.to_pickle(clustered_data_file)

In [ ]:

df_rt_rj

### Deviding the dataframe into RJ-files and RT-files

In [ ]:
df_rj = df_rt_rj[df_rt_rj['file_id'] == 'RJ'].drop('file_id', axis=1)
df_rt = df_rt_rj[df_rt_rj['file_id'] == 'RT'].drop('file_id', axis=1)

## Calculating the CV of the percentege of each cluster in the RJ-files

In [ ]:
df_rj_counts = df_rj.groupby(['batch', 'cluster']).size().reset_index(name='counts')

total_counts_rj = df_rj.groupby('batch').size().reset_index(name='total_counts')

merged_counts_rj = df_rj_counts.merge(total_counts_rj, on='batch')

merged_counts_rj['percentage'] = merged_counts_rj['counts'] / merged_counts_rj['total_counts'] * 100

pivot_data_rj = merged_counts_rj.pivot(index='cluster', columns='batch', values='percentage')
pivot_data_rj = pivot_data_rj.fillna(0)

pivot_data_rj

In [ ]:
cv_clusters_rj = pd.DataFrame()

cv_clusters_rj['mean'] = pivot_data_rj.mean(axis=1)
cv_clusters_rj['std'] = pivot_data_rj.std(axis=1)

cv_clusters_rj['CV'] = cv_clusters_rj['std'] / cv_clusters_rj['mean']

cv_clusters_rj.describe()

# Plotting the CV 

In [ ]:
import plotly.express as px

fig_box_rj = px.box(cv_clusters_rj, y="CV",points="all")
fig_box_rj.show()

In [ ]:
total_cluster_size_rj = df_rj.groupby('cluster').size().reset_index(name='cluster_size')
rj_box_size = total_cluster_size_rj.merge(cv_clusters_rj, on='cluster')
rj_box_size['test'] = ['']*len(rj_box_size)

In [ ]:
fig_scatter_rj = px.scatter(rj_box_size, 
                 x='test',     
                 y='CV',         
                 size='cluster_size', 
                 hover_name='cluster') 

fig_scatter_rj.update_traces(marker=dict(line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig_scatter_rj.show()

In [ ]:
import plotly.graph_objects as go

fig_combined = go.Figure()

for trace in fig_box_rj.data:
    fig_combined.add_trace(trace)

for trace in fig_scatter_rj.data:
    fig_combined.add_trace(trace)

fig_combined.update_layout(
    title='Combined Box and Scatter Plot',
    xaxis_title='Control Sample (RJ) in Pool 1',
    yaxis_title='CV',
    showlegend=False
)


fig_combined.show()

# Integrating RT

In [ ]:
df_rt_counts = df_rt.groupby(['batch', 'cluster']).size().reset_index(name='counts')

total_counts_rt = df_rt.groupby('batch').size().reset_index(name='total_counts')

merged_counts_rt = df_rt_counts.merge(total_counts_rt, on='batch')

merged_counts_rt['percentage'] = merged_counts_rt['counts'] / merged_counts_rt['total_counts'] * 100

pivot_data_rt = merged_counts_rt.pivot(index='cluster', columns='batch', values='percentage')
pivot_data_rt = pivot_data_rt.fillna(0)

pivot_data_rt

In [ ]:
cv_clusters_rt = pd.DataFrame()

cv_clusters_rt['mean'] = pivot_data_rt.mean(axis=1)
cv_clusters_rt['std'] = pivot_data_rt.std(axis=1)

cv_clusters_rt['CV'] = cv_clusters_rt['std'] / cv_clusters_rt['mean']

cv_clusters_rt.describe()

In [ ]:
import plotly.express as px

fig_box_rt = px.box(cv_clusters_rt, y="CV",points="all")
fig_box_rt.show()

In [ ]:
total_cluster_size_rt = df_rt.groupby('cluster').size().reset_index(name='cluster_size')
rt_box_size = total_cluster_size_rt.merge(cv_clusters_rt, on='cluster')
rt_box_size['test'] = ['']*len(rt_box_size)

In [ ]:
fig_scatter_rt = px.scatter(rt_box_size, 
                 x='test',     
                 y='CV',          
                 size='cluster_size',  
                 hover_name='cluster')

fig_scatter_rt.update_traces(marker=dict(line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig_scatter_rt.show()

# Plotting all togehter

In [ ]:
import plotly.graph_objects as go


fig_combined = go.Figure()

for trace in fig_box_rj.data:
    modified_trace = trace
    modified_trace.x = ['RJ'] * len(trace.y) 
    fig_combined.add_trace(modified_trace)

for trace in fig_box_rt.data:
    modified_trace = trace
    modified_trace.x = ['RT'] * len(trace.y)  
    fig_combined.add_trace(modified_trace)

fig_combined.update_layout(
    title='Combined Box and Scatter Plot',
    xaxis_title='Validation Sample in Pool 1',
    yaxis_title='CV',
    showlegend=True 
)

fig_combined.show()


In [ ]:
import plotly.graph_objects as go



fig_combined = go.Figure()

for trace in fig_scatter_rj.data:
    modified_trace = trace
    modified_trace.x = ['RJ'] * len(trace.y)  
    fig_combined.add_trace(modified_trace)

for trace in fig_scatter_rt.data:
    modified_trace = trace
    modified_trace.x = ['RT'] * len(trace.y) 
    fig_combined.add_trace(modified_trace)

fig_combined.update_layout(
    title='Combined scatter and Scatter Plot',
    xaxis_title='Validation Sample in Pool 1',
    yaxis_title='CV',
    showlegend=True
    )

fig_combined.show()

In [ ]:
import plotly.graph_objects as go


fig_combined = go.Figure()

for trace in fig_box_rj.data:
    modified_trace = trace
    modified_trace.x = ['RJ'] * len(trace.y)  
    fig_combined.add_trace(modified_trace)

for trace in fig_box_rt.data:
    modified_trace = trace
    modified_trace.x = ['RT'] * len(trace.y)  
    modified_trace.marker.color = 'green' 
    modified_trace.line.color = 'green' 
    fig_combined.add_trace(modified_trace)

for trace in fig_scatter_rj.data:
    modified_trace = trace
    modified_trace.x = ['RJ '] * len(trace.y)  
    fig_combined.add_trace(modified_trace)

for trace in fig_scatter_rt.data:
    modified_trace = trace
    modified_trace.x = ['RT '] * len(trace.y)  
    modified_trace.marker.color = 'green'  
    fig_combined.add_trace(modified_trace)

fig_combined.update_layout(
    title='Combined Box and Scatter Plot',
    xaxis_title='Validation and Control Samples in Pool 1',
    yaxis_title='Coefficient of Variation (CV)',
    showlegend=True 
)

fig_combined.show()
